# DataArray with coordinates from YAML

This example demonstrates how to validate DataArrays with both dimension and non-dimension coordinates using a YAML schema.

The `schema.yaml` file defines:
- **dtype, name, shape, dims**: Basic DataArray properties
- **coords**: Coordinate validation schema
  - **lat, lon**: Dimension coordinates (1D arrays matching each dimension)
  - **time**: Non-dimension coordinate (scalar value)
  - **require_all_keys**: All specified coordinates must be present
  - **allow_extra_keys**: Extra coordinates not in schema are allowed

This example demonstrates two types of coordinates:

1. **Dimension coordinates**: Coordinates that share a name with a dimension (lat, lon)
2. **Non-dimension coordinates**: Additional coordinates that don't correspond to dimensions (time)

In [ ]:
# Basic imports
import numpy as np
import xarray as xr

from xarray_validate import DataArraySchema, SchemaError

# Load schema from YAML file
schema = DataArraySchema.from_yaml("schema.yaml")

In [ ]:
# Create latitude and longitude coordinates
lat = np.linspace(-89.5, 89.5, 180)
lon = np.linspace(-179.5, 179.5, 360)

# Create a DataArray with coordinates that match the schema
data = xr.DataArray(
    np.random.randn(180, 360),
    dims=["lat", "lon"],
    coords={
        "lat": lat,
        "lon": lon,
        "time": np.datetime64("2024-01-01").astype("int64"),
    },
    name="sea_surface_temperature",
)

# Validate the DataArray
print("Validating DataArray with coordinates...")
result = schema.validate(data)
print("Validation passed!")

In [ ]:
# Add an extra coordinate (should still pass because allow_extra_keys=true)
print("\nAdding extra coordinate 'depth'...")
data_with_extra = data.assign_coords(depth=100.0)
result = schema.validate(data_with_extra)
print("Validation passed (extra coordinates allowed)!")

In [ ]:
# Try with missing coordinate
print("\nRemoving required coordinate 'time'...")
data_missing = data.drop_vars("time")
try:
    schema.validate(data_missing)
except SchemaError as e:
    print(f"Validation failed as expected: {e}")

In [ ]:
# Try with wrong coordinate shape
print("\nTrying with wrong latitude shape...")
wrong_coords = xr.DataArray(
    np.random.randn(90, 360),
    dims=["lat", "lon"],
    coords={
        "lat": np.linspace(-89.5, 89.5, 90),  # Wrong shape: 90 instead of 180
        "lon": lon,
        "time": np.datetime64("2024-01-01").astype("int64"),
    },
    name="sea_surface_temperature",
)
try:
    schema.validate(wrong_coords)
except SchemaError as e:
    print(f"Validation failed as expected: {e}")